In [15]:
import pandas as pd
import numpy as np
import sklearn
import pickle
from sklearn import preprocessing
from sklearn import model_selection as ms
from sklearn import linear_model

In [16]:
# Đọc file csv, xóa cột số thứ tự và cột AveMonthSpend
csv = pd.read_csv('CSV.csv').drop(['Unnamed: 0','BikeBuyer','CustomerID'], axis=1)
#csv = pd.read_csv('CSV.csv').drop(['Unnamed: 0'], axis=1)
csv.head()

,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,Age
0,Bachelors,Professional,M,M,1,0,0,2,137947,89,"25,45"
1,Bachelors,Professional,M,S,0,1,3,3,101141,117,"25,45"
2,Bachelors,Professional,M,M,1,1,3,3,91945,123,"25,45"
3,Bachelors,Professional,F,S,0,1,0,0,86688,50,"25,45"
4,Bachelors,Professional,F,S,1,4,5,5,92771,95,"25,45"


In [27]:
# Tạo label
labels = np.array(csv['AveMonthSpend'])
print (labels)

[ 89 117 123 ...  79  65  68]


In [31]:
# Chuyển các dữ liệu dạng biến phân loại về dạng binary dummy, sử dụng one hot code 
def encode_string(cat_feature, name_col):
    enc = preprocessing.LabelEncoder()
    enc = enc.fit(cat_feature)
    pickle.dump(enc, open('encode' + name_col + '.sav', 'wb'))  
    enc_cat_feature = enc.transform(cat_feature)
    
    ohe = preprocessing.OneHotEncoder(categories='auto')
    return ohe.fit_transform(enc_cat_feature.reshape(-1,1)).toarray()

col_to_encodes = ['Occupation','Gender','MaritalStatus',
                  'HomeOwnerFlag','Age']
features = encode_string(csv['Education'], 'Education')
for col in col_to_encodes:
    tmp = encode_string(csv[col], col)
    features = np.concatenate([features, tmp], axis = 1)
features.shape

(16404, 20)

In [21]:
# Thêm các giá trị số vào features 
features = np.concatenate([features, np.array(csv[['NumberCarsOwned','NumberChildrenAtHome',
                                                   'TotalChildren','YearlyIncome']])], axis = 1)
print (features.shape)

(16404, 24)


In [22]:
# Tách thành 2 tập train và test
indx = range(features.shape[0])
indx = ms.train_test_split(indx, test_size = 4000)
x_train = features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
x_test = features[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [32]:
# Chuẩn hóa z-score cho các trường
zscore = preprocessing.StandardScaler()
x_train[:,20:] = zscore.fit_transform(x_train[:,20:])
x_test[:,20:] = zscore.fit_transform(x_test[:,20:])

In [ ]:
zscore

In [33]:
# Áp dụng mô hình logistic 
linear_mode = linear_model.LinearRegression()
linear_mode.fit(x_train, y_train)
pickle.dump(linear_mode, open('final_linearregression.sav', 'wb'))  

In [34]:
score_proba = linear_mode.predict(x_test)


In [35]:
from sklearn import metrics as sklm
import math
def print_metrics(y_true, y_predicted, n_parameters):
    ## First compute R^2 and the adjusted R^2
    r2 = sklm.r2_score(y_true, y_predicted)
    r2_adj = r2 - (n_parameters - 1)/(y_true.shape[0] - n_parameters) * (1 - r2)
    
    ## Print the usual metrics and the R^2 values
    print('Mean Square Error      = ' + str(sklm.mean_squared_error(y_true, y_predicted)))
    print('Root Mean Square Error = ' + str(math.sqrt(sklm.mean_squared_error(y_true, y_predicted))))
    print('Mean Absolute Error    = ' + str(sklm.mean_absolute_error(y_true, y_predicted)))
    print('Median Absolute Error  = ' + str(sklm.median_absolute_error(y_true, y_predicted)))
    print('R^2                    = ' + str(r2))
    print('Adjusted R^2           = ' + str(r2_adj))
    
print_metrics(y_test, score_proba, 2)    

Mean Square Error      = 38.438946258544924
Root Mean Square Error = 6.199915020268014
Mean Absolute Error    = 4.72102734375
Median Absolute Error  = 3.65625
R^2                    = 0.9490309708174457
Adjusted R^2           = 0.9490182221858342


In [19]:
print(score_proba)

[ 57.8828125  46.296875   89.2890625 ...  84.734375   70.7578125
 119.34375  ]


In [20]:
point = pd.DataFrame(score_proba)
print (point.shape)
point.to_csv('KQ.csv')

(4000, 1)
